In [1]:
import numpy as np
import sys
import os
import time
import re
from collections import Counter, OrderedDict
from tqdm import tqdm

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import tensorflow as tf

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer


import torchtext
from torchtext.data import get_tokenizer
from torchtext.vocab import vocab
from torchtext.data.functional import to_map_style_dataset

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
from torch.optim.lr_scheduler import StepLR

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import os, shutil
data_list = []
folder = '/content/drive/MyDrive/Study/ML/dataset'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    with open(file_path, 'r') as f:
        text = f.read()
        #text = " ".join([x.lower() for x in re.split('\s+', text)])
        data_list.append(text.lower())
data = " ".join(data_list)
text = re.sub('\n', ' ', data)

In [3]:
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(text)
counter = Counter(tokens)
sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)
vocabulary = vocab(ordered_dict)
unk_token = '<unk>'
vocabulary.insert_token(unk_token, 0)
vocabulary.set_default_index(0)

In [4]:
print(len(text.split()))
print(len(tokens))

29438
29479


In [5]:
print(text[:250])
print(tokens[:10])

my dearest girl, i have been a walk this morning with a book in my hand, but as usual i have been occupied with nothing but you: i wish i could  say in an agreeable manner. i am tormented day and night. they talk of my going to italy. ‘tis certain i 
['my', 'dearest', 'girl', 'i', 'have', 'been', 'a', 'walk', 'this', 'morning']


In [6]:
len(vocabulary)

3652

In [7]:
# Reference:
# "https://towardsdatascience.com/multiclass-text-classification-using-lstm-in-pytorch-eac56baed8df"

glove_path = '/content/drive/MyDrive/Study/ML/glove.6B.100d.txt'

# read GloVe data
def load_glove_vectors(glove_file):
    word_vectors = {}
    with open(glove_file) as f:
        for line in f:
            split = line.split()
            word_vectors[split[0]] = np.array([float(x) for x in split[1:]])
    return word_vectors

# create embedding matrix
def get_emb_matrix(pretrained, vocab, emb_size = 100):
    vocab_size = len(vocab)
    W = np.zeros((vocab_size, emb_size), dtype="float32")
    for word in vocab.get_itos():
        if word in pretrained:
            W[vocab[word]] = pretrained[word]
        else:
            W[vocab[word]] = np.random.uniform(-0.25,0.25, emb_size)  
    return W

word_vecs = load_glove_vectors(glove_path)
pretrained_weights = get_emb_matrix(word_vecs, vocabulary)

In [8]:
def create_dataset(text_list, max_len=400):
    labels, texts = [], []
    for i in range(len(text_list)-3):
        w1 = vocabulary[text_list[i]]
        w2 = vocabulary[text_list[i+1]]
        w3 = vocabulary[text_list[i+2]]
        w4 = vocabulary[text_list[i+3]]
        texts.append([w1, w2, w3])
        labels.append(w4)
    return TensorDataset(torch.from_numpy(np.array(texts)), torch.from_numpy(np.array(labels)))

In [9]:
train_split = int(len(tokens)*0.8)
test_split = len(tokens)-train_split
train_dataset = create_dataset(tokens[:train_split])
test_dataset = create_dataset(tokens[-test_split:])

In [10]:
print(len(train_dataset))

23580


In [36]:
BATCH_SIZE = 150
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [65]:
class LSTM_Glove(nn.Module):
    ''' LSTM model with GloVe embeddings'''

    def __init__(self, vocab_size, embedding_dim, hidden_dim, no_layers, glove_weights):
        super().__init__()

        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(glove_weights), padding_idx=0, freeze=False)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, no_layers, batch_first=True)
        self.linear1 = nn.Linear(hidden_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, vocab_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x, (ht, ct) = self.lstm(x)
        x = self.linear2(ht[-1])
        #x = self.relu(x)
        #x = self.linear2(x)
        x = self.softmax(x)
        return x

In [38]:
def acc(prediction, label):
    pred = torch.argmax(prediction, dim=1)
    return torch.sum(pred == label.squeeze()).item()

In [39]:
def train_step(model, loader, loss_fn, optimizer, device):
    ''' Calculate loss and accuracy for one epoch'''
    train_losses = []
    total_acc, total_count = 0, 0
    for batch_idx, (text, label) in enumerate(loader):

        text = text.to(device)
        label = label.to(device)

        prediction = model(text)

        loss_value = loss_fn(prediction, label)
        loss_value.backward()
        train_losses.append(loss_value.item())
        
        optimizer.step()
        optimizer.zero_grad()
        total_acc += acc(prediction, label)
        total_count += label.size(0)
  
    return np.mean(train_losses), total_acc/total_count

In [15]:
def val_step(model, loader, device):
    ''' Calculate accuracy for one epoch'''
    total_acc, total_count = 0, 0
    with torch.no_grad():
      y_pred, y_true = [], []
      for batch_idx, (text, label) in enumerate(loader):          
          text = text.to(device) 
          label = label.to(device)
          prediction = model(text)
          total_acc += acc(prediction, label)
          total_count += label.size(0)
    return total_acc/total_count

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [66]:
# define parameters
VOCAB_SIZE = len(vocabulary)
NO_LAYERS = 1
EMBED_DIM = 100
HIDDEN_DIM_1 = 70
HIDDEN_DIM_2 = 50
EPOCHS = 20

In [73]:
# initialize a model with trainable embeddings
model_emb = LSTM_Glove(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM_1, NO_LAYERS, pretrained_weights).to(device)
lr=0.005
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_emb.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=50, gamma=0.5)

In [74]:
# train with training data and validate with validation data
EPOCHS = 300
for epoch in tqdm(range(EPOCHS)):
    model_emb.train()
    train_loss, train_acc = train_step(model_emb, train_loader, criterion, optimizer, device)
    print(f"Epochs = {epoch}, Training Loss : {train_loss}; Training Accuracy: {train_acc}.")
    model_emb.eval()
    val_acc = val_step(model_emb, valid_loader, device)
    print(f"Epochs = {epoch}, Validation Accuracy : {val_acc}")
    #scheduler.step()

  0%|          | 1/300 [00:00<03:29,  1.43it/s]

Epochs = 0, Training Loss : 8.16157250464717; Training Accuracy: 0.046055979643765906.
Epochs = 0, Validation Accuracy : 0.06159850670286781


  1%|          | 2/300 [00:01<03:36,  1.38it/s]

Epochs = 1, Training Loss : 8.155993678901769; Training Accuracy: 0.04775233248515691.
Epochs = 1, Validation Accuracy : 0.06159850670286781


  1%|          | 3/300 [00:02<03:33,  1.39it/s]

...

Epochs = 297, Training Loss : 7.901748023455656; Training Accuracy: 0.30318066157760815.
Epochs = 297, Validation Accuracy : 0.16205667741388088


100%|█████████▉| 299/300 [03:39<00:00,  1.35it/s]

Epochs = 298, Training Loss : 7.901761987541295; Training Accuracy: 0.3024173027989822.
Epochs = 298, Validation Accuracy : 0.1629051416935347


100%|██████████| 300/300 [03:39<00:00,  1.36it/s]

Epochs = 299, Training Loss : 7.902406363547603; Training Accuracy: 0.30305343511450383.
Epochs = 299, Validation Accuracy : 0.16409299168505007


In [20]:
def continue_seq(text):
    tokens = tokenizer.tokenize(text)
    w1, w2, w3 = vocabulary[tokens[-3]], vocabulary[tokens[-2]], vocabulary[tokens[-1]]
    t = torch.IntTensor([w1, w2, w3]).to(device)
    pred = model_emb.forward(torch.reshape(t, (1, 3)))
    a = torch.argmax(pred, dim=1)
    new_word = vocabulary.lookup_token(a.item())
    new_text=text+' '+new_word
    return new_text
    # if new_word=='.':
    #     return new_text
    # else:
    #     return continue_seq(new_text)

In [77]:
#start = 'i love you'
start = 'i write you to say that'
for i in range(15):
    start = continue_seq(start)
print(start)

i write you to say that i have you i have a of you and i have a of you and


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
